In [1]:
%pip install boto3
%pip install pandas
%pip install requests
%config Completer.use_jedi = False

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\alexp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\alexp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\alexp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from pathlib import Path
import csv
import pandas as pd
import requests
import base64
import boto3
from datetime import datetime
import os

## Initial configuration

In [3]:
path_to_configuration = r'C:\Users\alexp\Development\AWS-expense-analyzer\ignored\jupyter_configuration.csv'
config = pd.read_csv(path_to_configuration)
config['key']

0                  api_path
1    path_to_bank_statement
2                 bank_type
Name: key, dtype: object

In [4]:
def get_value_from_config(key):
    return config.loc[config['key'] == key, 'value'].values[0]

In [5]:
path_to_bank_statement = get_value_from_config('path_to_bank_statement')
bank_type = get_value_from_config('bank_type')
path_to_bank_statement

'C:\\Users\\alexp\\Development\\AWS-expense-analyzer\\AccountStatementParserLambda\\src\\main\\resources\\bankStatements\\pkobp.csv'

### Utils

In [6]:
text_encoding = 'Windows-1250'

def encode_base64(text):
    text_bytes = text.encode(text_encoding)
    text_bytes_base64 = base64.b64encode(text_bytes)
    return text_bytes_base64.decode(text_encoding)

def decode_base64(text):
    text_bytes = text.encode(text_encoding)
    text_bytes_base64 = base64.b64decode(text_bytes)
    return text_bytes_base64.decode(text_encoding)

### Configure endpoints

In [7]:
def get_path_to_endpoint(endpoint):
    return get_value_from_config('api_path') + endpoint

In [8]:
test_endpoint = get_path_to_endpoint('test')
parse_bank_statements_endpoint = get_path_to_endpoint('transactions/parser') # AccountStatementParserLambda

## Read bank statement

In [9]:
bank_statement_content = Path(path_to_bank_statement).read_text()
bank_statement_content_base64 = encode_base64(bank_statement_content)

## Upload bank statement to AWS

In [21]:
current_date = datetime.today().strftime('%Y-%m-%d')
bucket_name = 'expenses-counter-bank-statements'
filename = os.path.basename(path_to_bank_statement)
s3_filename = '{}/{}-{}'.format(current_date, bank_type, filename)
s3_filename

'2023-08-11/PKO_BP-pkobp.csv'

In [25]:
s3 = boto3.client('s3')
result = None
with open(path_to_bank_statement, "rb") as file:
    s3.upload_fileobj(file, bucket_name, s3_filename)

### Test connection

In [26]:
response = requests.get(test_endpoint)
response.text

'Jest git'

### Map bank statements to transactions

In [102]:
params = {
    "accountStatement": bank_statement_content_base64,
    "bankType": bank_type
}

In [103]:
headers = {'content-type': 'application/json'}
response = requests.get(parse_bank_statements_endpoint, params=params, headers=headers)
response.status_code

414